In [ ]:
# GENERAL IMPORTS

import pandas as pd
import numpy as np
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='lnumerow', api_key='mDhzegrhH0qf8rlFbzOo')
plotly.offline.init_notebook_mode(connected=True)

import datetime
import math

In [ ]:
datafile = r'''Data\IVE_bidask1min.txt'''

colnames = ['Date', 'Time', 'BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskLow', 'AskHigh', 'AskClose']
fullpricedata = pd.read_csv(datafile, names=colnames)

fullpricedata['DateTime'] = (fullpricedata['Date']+fullpricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
del fullpricedata['Date']
del fullpricedata['Time']

fullpricedata = fullpricedata[[(dt >= datetime.datetime(dt.year, dt.month, dt. day, 9, 30, 0)) and (dt <= datetime.datetime(dt.year, dt.month, dt. day, 16, 0, 0)) for dt in fullpricedata['DateTime']]].copy()

pricedata = fullpricedata[fullpricedata['DateTime'] > datetime.datetime(2019,2,10,0,0,0)].copy()

In [ ]:
def candlestickChart():
    
    data = [ dict(
            type = 'candlestick',
            x = [ii for ii in range(len(pricedata['BidLow']))],
            low = pricedata['BidLow'],
            open = pricedata['BidOpen'],
            close = pricedata['BidClose'],
            high = pricedata['BidHigh'])
           ]

    layout = dict(
            title = 'Candlestick Chart',
            xaxis = dict(
                title = dict(
                    text = 'Time'
                )
            ),
            yaxis = dict(
                title = dict(
                    text = 'Price'
                )
            ))
        
    return go.FigureWidget( data=data, layout=layout)

candle = candlestickChart()
candle

In [ ]:
cash = {'High':10000,'Low':10000,'Open':10000,'Close':10000}
shares = {'High':0,'Low':0,'Open':0,'Close':0}
shares_value = {'High':0,'Low':0,'Open':0,'Close':0}
net_value = dict(cash)

cash_list = list()
shares_list = list()
shares_value_list = list()
net_value_list = list()

start_idx = pricedata.first_valid_index();
commission = 0
threshold = 0.01
scale = 100

for index, row in pricedata.iterrows():
    buy = 0
    sell = 0
    
    if index > start_idx:
        print(index)
        last_row = pricedata.loc[index-1]
    else:
        print(index)
        last_row = pricedata.loc[index]
        
    delta = row['BidOpen'] - last_row['BidClose']
    
    if delta > threshold:
        buy = 1
        quantity = scale*round(100*delta)
    elif delta < -threshold:
        sell = 1
        quantity = scale*round(-100*delta)
    
    for k in cash.keys():
        bid_price = row['Bid' + str(k)]
        ask_price = row['Ask' + str(k)]
        if buy and cash[k] > commission:
            print('Buy!')
            cost = quantity*ask_price
            cost = min(cost, cash[k])
            quantity = cost/ask_price
            cash[k] -= cost
            shares[k] += quantity
        elif sell and shares[k] > 0:
            print('Sell!')
            quantity = min(quantity, shares[k])
            cash[k] += quantity*bid_price
            shares[k] -= quantity
        cash[k] -= commission
        
        # Careful with this rounding
        cash[k] = round(cash[k]*100)/100
        shares_value[k] = round(bid_price*shares[k])
        net_value[k] = round(cash[k] + shares_value[k])
        
    cash_list.append(dict(cash))
    shares_list.append(dict(shares))
    shares_value_list.append(dict(shares_value))
    net_value_list.append(dict(net_value))        
    print('Cash: ' + str(cash['Open']) + ', Shares: ' + str(shares['Open']) + ', Net Value: ' + str(net_value['Open']))

In [ ]:
cash_open = [d['Open'] for d in cash_list]
shares_open = [d['Open'] for d in shares_list]
shares_value_open = [d['Open'] for d in shares_value_list]
net_value_open = [d['Open'] for d in net_value_list]

In [ ]:
def profitChart():
    
    data = [ dict(
            type = 'scatter',
            x = [ii for ii in range(len(pricedata['BidLow']))],
            y = cash_open),
            
            dict(
            type = 'scatter',
            x = [ii for ii in range(len(pricedata['BidLow']))],
            y = shares_value_open),
            
            dict(
            type = 'scatter',
            x = [ii for ii in range(len(pricedata['BidLow']))],
            y = net_value_open)
           ]

    layout = dict(
            title = 'Profit Chart',
            xaxis = dict(
                title = dict(
                    text = 'Time'
                )
            ),
            yaxis = dict(
                title = dict(
                    text = 'Price'
                )
            ))
        
    return go.FigureWidget( data=data, layout=layout)

In [ ]:
def profitChart2():
    
    data = [
            dict(
            type = 'scatter',
            x = [ii for ii in range(len(pricedata['BidLow']))],
            y = [nv - net_value_open[0] for nv in net_value_open])
           ]

    layout = dict(
            title = 'Profit Chart',
            xaxis = dict(
                title = dict(
                    text = 'Time'
                )
            ),
            yaxis = dict(
                title = dict(
                    text = 'Price'
                )
            ))
        
    return go.FigureWidget( data=data, layout=layout)

In [ ]:
p=profitChart()
p2=profitChart2()

import ipywidgets
ipywidgets.VBox([p,p2])